In [ ]:
import glob
from pathlib import Path
from tempfile import gettempdir

from common import *

NUM_N = 1
NUM_F = 1
SHOW_E = True
SHOW_Q = True

PARTICLES = ["H"]
PARTICLES.extend([f"N{i + 1}" for i in range(NUM_N)])
PARTICLES.extend([f"L{i + 1}" for i in range(NUM_F)])
if SHOW_E:
    PARTICLES.extend([f"e{i + 1}" for i in range(NUM_F)])
if SHOW_Q:
    PARTICLES.extend([f"Q{i + 1}" for i in range(NUM_F)])
    PARTICLES.extend([f"u{i + 1}" for i in range(NUM_F)])
    PARTICLES.extend([f"d{i + 1}" for i in range(NUM_F)])

display(HTML(f"<h1>{NUM_N} Neutrino, {NUM_F} Generations</h1>"))

# Find the default output directory
OUTPUT_DIR = Path(gettempdir()) / "boltzmann-solver" / "full"
display(HTML(f"<span>Loading data from <code>{OUTPUT_DIR}</code></span>"))


 ## Decay Only

In [ ]:
data = read_number_density(OUTPUT_DIR / f"decay_n{NUM_N}f{NUM_F}.csv")

display(plot_integration(data))
display(plot_densities(data, PARTICLES))


 ## Washout

In [ ]:
data = read_number_density(OUTPUT_DIR / f"washout_n{NUM_N}f{NUM_F}.csv")

display(plot_integration(data))
display(plot_densities(data, PARTICLES))


 ## Full

In [ ]:
data = read_number_density(OUTPUT_DIR / f"full_n{NUM_N}f{NUM_F}.csv")

display(plot_integration(data))
display(plot_densities(data, PARTICLES))


 ## Miscellaneous

 ### Evolution

In [ ]:
data, ptcls = read_evolution(OUTPUT_DIR / f"evolution_n{NUM_N}f{NUM_F}.json")

display(
    go.Figure(
        data=[go.Scatter(name=p, x=data["beta"], y=data[p, "mass"]) for p in ptcls],
        layout=go.Layout(
            xaxis=go.layout.XAxis(
                title="Inverse Temperature [GeV⁻¹]",
                type="log",
                exponentformat="power",
            ),
            yaxis=go.layout.YAxis(
                title="Mass [GeV]", type="log", exponentformat="power"
            ),
        ),
    )
)

display(
    go.Figure(
        data=[
            go.Scatter(name=p, x=data["beta"], y=data[p, "mass"] * data["beta"])
            for p in ptcls
        ],
        layout=go.Layout(
            xaxis=go.layout.XAxis(
                title="Inverse Temperature [GeV⁻¹]",
                type="log",
                exponentformat="power",
            ),
            yaxis=go.layout.YAxis(
                title="Mass / Temperatre", type="log", exponentformat="power"
            ),
        ),
    )
)

display(
    go.Figure(
        data=[
            go.Scatter(name=p, x=data["beta"], y=data[p, "width"] / data[p, "mass"])
            for p in ptcls
        ],
        layout=go.Layout(
            xaxis=go.layout.XAxis(
                title="Inverse Temperature [GeV⁻¹]",
                type="log",
                exponentformat="power",
            ),
            yaxis=go.layout.YAxis(
                title="Width / Mass", type="log", exponentformat="power"
            ),
        ),
    )
)


 ### Higgs Equilibrium

In [ ]:
data = list(
    map(
        lambda f: read_number_density(f, quiet=True),
        sorted(glob.glob(str(OUTPUT_DIR / "higgs_equilibrium" / "*.csv"))),
    )
)

go.Figure(
    data=[
        go.Scatter(
            x=df["beta"],
            y=df["n-H"],
            mode="lines",
            line=go.scatter.Line(
                color=plotly.colors.sample_colorscale("Viridis", i / len(data))[0]
            ),
            showlegend=False,
        )
        for i, df in enumerate(data)
    ]
    + [
        go.Scatter(
            x=df["beta"],
            y=df["eq-H"],
            mode="lines",
            line=go.scatter.Line(color="black"),
            showlegend=False,
        )
        for i, df in enumerate(data)
    ],
    layout=go.Layout(
        xaxis=go.layout.XAxis(
            title="Inverse Temperature [GeV⁻¹]",
            type="log",
            exponentformat="power",
        ),
        yaxis=go.layout.YAxis(
            title="Normalized Number Density", type="log", exponentformat="power"
        ),
        width=1000,
    ),
)


 ### Lepton Equilibrium

In [ ]:
data = list(
    map(
        lambda f: read_number_density(f, quiet=True),
        sorted(glob.glob(str(OUTPUT_DIR / "lepton_equilibrium" / "*.csv"))),
    )
)

go.Figure(
    data=[
        go.Scatter(
            x=df["beta"],
            y=df["n-L1"],
            mode="lines",
            line=go.scatter.Line(
                color=plotly.colors.sample_colorscale("Viridis", i / len(data))[0]
            ),
            showlegend=False,
        )
        for i, df in enumerate(data)
    ]
    + [
        go.Scatter(
            x=df["beta"],
            y=df["eq-L1"],
            mode="lines",
            line=go.scatter.Line(color="black"),
            showlegend=False,
        )
        for i, df in enumerate(data)
    ],
    layout=go.Layout(
        xaxis=go.layout.XAxis(
            title="Inverse Temperature [GeV⁻¹]",
            type="log",
            exponentformat="power",
        ),
        yaxis=go.layout.YAxis(
            title="Normalized Number Density", type="log", exponentformat="power"
        ),
        width=1000,
    ),
)


 ### Gammas

In [ ]:
data = pd.read_csv(OUTPUT_DIR / "gamma.csv")
display(plot_gamma(data))


In [ ]:
data = pd.read_csv(OUTPUT_DIR / "asymmetry.csv")
display(plot_gamma(data))
